## Lab | Cross Validation

**Scenario**

For this lab, we will build a model on customer churn binary classification problem. You will be using files_for_lab/Customer-Churn.csv file.

In [74]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score 

In [75]:
churnData = pd.read_csv("Customer-Churn.csv")
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [76]:
churnData.shape

(7043, 16)

In [77]:
churnData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   OnlineSecurity    7043 non-null   object 
 7   OnlineBackup      7043 non-null   object 
 8   DeviceProtection  7043 non-null   object 
 9   TechSupport       7043 non-null   object 
 10  StreamingTV       7043 non-null   object 
 11  StreamingMovies   7043 non-null   object 
 12  Contract          7043 non-null   object 
 13  MonthlyCharges    7043 non-null   float64
 14  TotalCharges      7043 non-null   object 
 15  Churn             7043 non-null   object 
dtypes: float64(1), int64(2), object(13)
memory

In [78]:
churnData.isna().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [79]:
churnData["TotalCharges"] = churnData["TotalCharges"].apply(lambda x: np.nan if x == " " else x)

In [80]:
churnData.isna().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [81]:
# due to small numbers of null values, dropping a null values from Total Charges column
churnData.dropna(subset=['TotalCharges'], inplace=True)

In [82]:
churnData.shape

(7032, 16)

In [83]:
churnData.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [84]:
churnData.columns = [churnData.columns[i].lower() for i in range(len(churnData.columns))]

In [85]:
churnData.columns

Index(['gender', 'seniorcitizen', 'partner', 'dependents', 'tenure',
       'phoneservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection',
       'techsupport', 'streamingtv', 'streamingmovies', 'contract',
       'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [86]:
# Converting total charges column to numeric
churnData['totalcharges']= pd.to_numeric(churnData['totalcharges'], errors = 'coerce' )

In [87]:
churnData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7032 non-null   object 
 1   seniorcitizen     7032 non-null   int64  
 2   partner           7032 non-null   object 
 3   dependents        7032 non-null   object 
 4   tenure            7032 non-null   int64  
 5   phoneservice      7032 non-null   object 
 6   onlinesecurity    7032 non-null   object 
 7   onlinebackup      7032 non-null   object 
 8   deviceprotection  7032 non-null   object 
 9   techsupport       7032 non-null   object 
 10  streamingtv       7032 non-null   object 
 11  streamingmovies   7032 non-null   object 
 12  contract          7032 non-null   object 
 13  monthlycharges    7032 non-null   float64
 14  totalcharges      7032 non-null   float64
 15  churn             7032 non-null   object 
dtypes: float64(2), int64(2), object(12)
memory

 - **Encoding categorical**

In [88]:
cat = churnData[churnData.select_dtypes("object").columns]
cat.head()

,gender,partner,dependents,phoneservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,churn
0,Female,Yes,No,No,No,Yes,No,No,No,No,Month-to-month,No
1,Male,No,No,Yes,Yes,No,Yes,No,No,No,One year,No
2,Male,No,No,Yes,Yes,Yes,No,No,No,No,Month-to-month,Yes
3,Male,No,No,No,Yes,No,Yes,Yes,No,No,One year,No
4,Female,No,No,Yes,No,No,No,No,No,No,Month-to-month,Yes


In [89]:
cat.columns

Index(['gender', 'partner', 'dependents', 'phoneservice', 'onlinesecurity',
       'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
       'streamingmovies', 'contract', 'churn'],
      dtype='object')

In [90]:
cat['gender'].value_counts()

Male      3549
Female    3483
Name: gender, dtype: int64

In [91]:
cat['partner'].value_counts()

No     3639
Yes    3393
Name: partner, dtype: int64

In [92]:
cat['dependents'].value_counts()

No     4933
Yes    2099
Name: dependents, dtype: int64

In [93]:
cat['phoneservice'].value_counts()

Yes    6352
No      680
Name: phoneservice, dtype: int64

In [94]:
cat['onlinesecurity'].value_counts()

No                     3497
Yes                    2015
No internet service    1520
Name: onlinesecurity, dtype: int64

In [95]:
cat['onlinebackup'].value_counts()

No                     3087
Yes                    2425
No internet service    1520
Name: onlinebackup, dtype: int64

In [96]:
cat['deviceprotection'].value_counts()

No                     3094
Yes                    2418
No internet service    1520
Name: deviceprotection, dtype: int64

In [97]:
cat['techsupport'].value_counts()

No                     3472
Yes                    2040
No internet service    1520
Name: techsupport, dtype: int64

In [98]:
cat['streamingtv'].value_counts()

No                     2809
Yes                    2703
No internet service    1520
Name: streamingtv, dtype: int64

In [99]:
cat['streamingmovies'].value_counts()

No                     2781
Yes                    2731
No internet service    1520
Name: streamingmovies, dtype: int64

In [100]:
cat['contract'].value_counts()

Month-to-month    3875
Two year          1685
One year          1472
Name: contract, dtype: int64

In [101]:
cat['churn'].value_counts()

No     5163
Yes    1869
Name: churn, dtype: int64

**Below columns are type of binary values yes and no, so converting those columns into 0 and 1**
 - partner
 - dependents
 - phoneservice

In [102]:
churnData['partner'] = churnData['partner'].apply(lambda x: 1 if x == "Yes" else 0)

In [103]:
churnData['partner'].value_counts()

0    3639
1    3393
Name: partner, dtype: int64

In [104]:
churnData['dependents'] = churnData['dependents'].apply(lambda x: 1 if x == "Yes" else 0)

In [105]:
churnData['dependents'].value_counts()

0    4933
1    2099
Name: dependents, dtype: int64

In [106]:
churnData['phoneservice'] = churnData['phoneservice'].apply(lambda x: 1 if x == "Yes" else 0)

In [107]:
churnData['phoneservice'].value_counts()

1    6352
0     680
Name: phoneservice, dtype: int64

**Converting other categorical columns into numerical using get dummies**

In [108]:
cat_dummies = cat[['onlinesecurity',
       'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
       'streamingmovies', 'contract']]

In [109]:
cat_dummies.head(20)

,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract
0,No,Yes,No,No,No,No,Month-to-month
1,Yes,No,Yes,No,No,No,One year
2,Yes,Yes,No,No,No,No,Month-to-month
3,Yes,No,Yes,Yes,No,No,One year
4,No,No,No,No,No,No,Month-to-month
5,No,No,Yes,No,Yes,Yes,Month-to-month
6,No,Yes,No,No,Yes,No,Month-to-month
7,Yes,No,No,No,No,No,Month-to-month
8,No,No,Yes,Yes,Yes,Yes,Month-to-month
9,Yes,Yes,No,No,No,No,One year


In [110]:
cat_dummies = pd.get_dummies(cat_dummies)

In [111]:
cat_dummies.head(20)

,onlinesecurity_No,onlinesecurity_No internet service,onlinesecurity_Yes,onlinebackup_No,onlinebackup_No internet service,onlinebackup_Yes,deviceprotection_No,deviceprotection_No internet service,deviceprotection_Yes,techsupport_No,...,techsupport_Yes,streamingtv_No,streamingtv_No internet service,streamingtv_Yes,streamingmovies_No,streamingmovies_No internet service,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year
0,1,0,0,0,0,1,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
1,0,0,1,1,0,0,0,0,1,1,...,0,1,0,0,1,0,0,0,1,0
2,0,0,1,0,0,1,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
3,0,0,1,1,0,0,0,0,1,0,...,1,1,0,0,1,0,0,0,1,0
4,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
5,1,0,0,1,0,0,0,0,1,1,...,0,0,0,1,0,0,1,1,0,0
6,1,0,0,0,0,1,1,0,0,1,...,0,0,0,1,1,0,0,1,0,0
7,0,0,1,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
8,1,0,0,1,0,0,0,0,1,0,...,1,0,0,1,0,0,1,1,0,0
9,0,0,1,0,0,1,1,0,0,1,...,0,1,0,0,1,0,0,0,1,0


In [112]:
churnData['gender'].value_counts()

Male      3549
Female    3483
Name: gender, dtype: int64

In [113]:
# Converting male and female into 0 and 1
churnData['gender'] = churnData['gender'].apply(lambda x: 1 if x == "Male" else 0)

In [114]:
churnData['gender'].value_counts()

1    3549
0    3483
Name: gender, dtype: int64

In [115]:
# removing original calumns from which were transferred into gat dummies
churnData = churnData.drop(['onlinesecurity',
       'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
       'streamingmovies', 'contract'], axis =1)

In [116]:
churnData.head()

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,monthlycharges,totalcharges,churn
0,0,0,1,0,1,0,29.85,29.85,No
1,1,0,0,0,34,1,56.95,1889.50,No
2,1,0,0,0,2,1,53.85,108.15,Yes
3,1,0,0,0,45,0,42.30,1840.75,No
4,0,0,0,0,2,1,70.70,151.65,Yes


In [117]:
# concatinating all data into one
churnData_final = pd.concat([churnData, cat_dummies], axis=1)

In [118]:
churnData_final.head()

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,monthlycharges,totalcharges,churn,onlinesecurity_No,...,techsupport_Yes,streamingtv_No,streamingtv_No internet service,streamingtv_Yes,streamingmovies_No,streamingmovies_No internet service,streamingmovies_Yes,contract_Month-to-month,contract_One year,contract_Two year
0,0,0,1,0,1,0,29.85,29.85,No,1,...,0,1,0,0,1,0,0,1,0,0
1,1,0,0,0,34,1,56.95,1889.50,No,0,...,0,1,0,0,1,0,0,0,1,0
2,1,0,0,0,2,1,53.85,108.15,Yes,0,...,0,1,0,0,1,0,0,1,0,0
3,1,0,0,0,45,0,42.30,1840.75,No,0,...,1,1,0,0,1,0,0,0,1,0
4,0,0,0,0,2,1,70.70,151.65,Yes,1,...,0,1,0,0,1,0,0,1,0,0


In [119]:
# X-y split

X = churnData_final.drop("churn", axis=1)
y = churnData_final[["churn"]]

In [120]:
# Checking target columns values
y.value_counts()

churn
No       5163
Yes      1869
dtype: int64

### 1. Apply SMOTE for upsampling the data

In [121]:
smote = SMOTE()
X_sm, y_sm = smote.fit_resample(X, y)
y_sm.value_counts()

churn
No       5163
Yes      5163
dtype: int64

 - Use logistic regression to fit the model and compute the accuracy of the model.

In [122]:
# Train - Test split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.3, random_state=21)

In [123]:
LR = LogisticRegression(random_state=100, multi_class='ovr').fit(X_train, y_train)
s_predictions_lr = LR.predict(X_test)

LR.score(X_test, y_test)

0.8105229180116204

In [135]:
# Using cross-validation for logistic model
# K = 5
cross_val_scores_lr = cross_val_score(LR, X_train, y_train, cv=5, scoring ='accuracy')
print("Corss validation score is : ",cross_val_scores_lr)
print("Mean score is : ",cross_val_scores_lr.mean())

Corss validation score is :  [0.80982019 0.83540802 0.79806362 0.80761246 0.83044983]
Mean score is :  0.8162708246588849


 - Use decision tree classifier to fit the model and compute the accuracy of the model.

In [125]:
DT = DecisionTreeClassifier()
DT.fit(X_train, y_train)
s_predictions_DT = DT.predict_proba(X_test)

DT.score(X_test, y_test)

0.802775984506133

In [134]:
# Using cross-validation for Decision Tree model
# K = 5
cross_val_scores_DT = cross_val_score(DT, X_train, y_train, cv=5, scoring ='accuracy')
print("Corss validation score is : ",cross_val_scores_DT)
print("Mean score is : ",cross_val_scores_DT.mean())

Corss validation score is :  [0.79806362 0.80497925 0.78492393 0.783391   0.7916955 ]
Mean score is :  0.792610662033913


 - Compare the accuracies of the two models.
 
After comparing the two models accuracy, **Logistic Regression** model gives the more accurate predictions than the Decision Tree.

### 2.Apply TomekLinks for downsampling

In [127]:
# Applying imblearn.under_sampling.TomekLinks to the dataset.
from imblearn.under_sampling import TomekLinks
tl = TomekLinks(sampling_strategy='majority')
X_tl, y_tl = tl.fit_resample(X, y)
y_tl.value_counts()

churn
No       4595
Yes      1869
dtype: int64

In [128]:
X_tl2, y_tl2 = tl.fit_resample(X_tl, y_tl)
y_tl2.value_counts()

churn
No       4428
Yes      1869
dtype: int64

 - Use logistic regression to fit the model and compute the accuracy of the model.

In [132]:
lr_t = LogisticRegression(random_state=100, multi_class='ovr').fit(X_train, y_train)
t_predictions_lr = lr_t.predict(X_test)

In [133]:
lr_t.score(X_test, y_test)

0.8105229180116204

In [136]:
# Using cross-validation for logistic model
# K = 5
cross_val_scores_lr_t = cross_val_score(lr_t, X_train, y_train, cv=5, scoring ='accuracy')
print("Corss validation score is : ",cross_val_scores_lr_t)
print("Mean score is : ",cross_val_scores_lr_t.mean())

Corss validation score is :  [0.80982019 0.83540802 0.79806362 0.80761246 0.83044983]
Mean score is :  0.8162708246588849


 - Use decision tree classifier to fit the model and compute the accuracy of the model.

In [137]:
DT = DecisionTreeClassifier()
DT.fit(X_train, y_train)
s_predictions_DT = DT.predict_proba(X_test)

classification.score(X_test, y_test)

0.8105229180116204

In [138]:
# Using cross-validation for Decision Tree model
# K = 5
cross_val_scores_DT_t = cross_val_score(DT, X_train, y_train, cv=5, scoring ='accuracy')
print("Corss validation score is : ",cross_val_scores_DT_t)
print("Mean score is : ",cross_val_scores_DT_t.mean())

Corss validation score is :  [0.7966805  0.80705394 0.78838174 0.78477509 0.78546713]
Mean score is :  0.7924716794210973


 - Compare the accuracies of the two models.
 
Decision tree model gives the same accuracy as logistic reression model without the cross validation